In [35]:
import psycopg2

import pandas as pd
import numpy as np

from copy import deepcopy

import ast
import random
import networkx as nx
import time, unicodedata
import itertools

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

from joblib import Parallel, delayed

In [15]:
rc = pd.read_csv("../datasets/random_connections.csv",index_col="Unnamed: 0")
rc.sample(5)

from_user_id  to_user_id             formation
2067      152015204   285064188  {'2018.05.20': True}
37735      71784886   158635358  {'2018.05.21': True}
46608     950278803   147297022  {'2018.05.01': True}
24045       2425151   876789610  {'2018.05.07': True}
14057     895454951   827841372  {'2018.05.25': True}

In [19]:
import ast
str2dict = lambda d : ast.literal_eval(d)

In [21]:
rc.formation = rc.formation.apply(str2dict)

In [29]:
def present_in_date(changes_dates, queried_date):
    """
    checking if a connection is present in a queried date
    changes_dates: {d1:True, d2:False, d3:True} connection added or removed
    queried_date: e.g. "2018.05.08"
    """
    str2date = lambda strdate: datetime.strptime(strdate, '%Y.%m.%d')  # 2018.05.08
    changes = sorted(changes_dates,key=lambda d: str2date(d))
    queried_date = datetime.strptime(queried_date, '%Y.%m.%d')
    present = False
    for d in changes:
        if queried_date < str2date(d):
            break
        present = changes_dates[d]
    return present

In [30]:
def get_connections_by_date(date):
    nw = deepcopy(rc)
    for_col = nw.formation.apply(lambda dates: present_in_date(dates,date))
    return rc[for_col == True]

In [36]:
cons = get_connections_by_date('2018.05.07')

In [40]:
def clean(name, min_len=5, junk_replacement=''):
    try:
        cleaned = unicodedata.normalize('NFKD', name).encode('ascii', 'ignore').lower().decode("ascii")
    except TypeError:
        return junk_replacement
    if len(cleaned) < min_len:
        return junk_replacement
    return cleaned

In [41]:
def construct_network(connections, rec_metrics=True):
    G = nx.DiGraph()
    truncate = lambda x: int(str(int(x))[:9])
    for _, row in connections.iterrows():
        from_ = truncate (row["from_user_id"])
        to = truncate (row["to_user_id"])
        if from_ in twitter_users.index and to in twitter_users.index:
            G.add_edge(from_, to)

    augs = ["name", "screen_name", "match_name", "followers_count", "friends_count", "lang"]
    for node in G.nodes():
        user = twitter_users.loc[node]
        for aug in augs:
            if type(user[aug]) == str:
                m = clean(user[aug])
            else:
                m = user[aug]
            G.nodes[node][aug] = m

    if rec_metrics:
        return recalculate_metrics(G, parse=False)
    return G

In [45]:
twitter_users = pd.read_csv("../REST/static/twitter_users.csv", index_col="id")
g = construct_network(cons, False)

In [46]:
g.nodes()

NodeView((99528248, 280162312, 825982985, 214511627, 174575628, 429101069, 492441614, 282013711, 309387281, 878145554, 378589207, 219985599, 166260766, 521494559, 252747814, 951803948, 846204973, 477970480, 263102513, 303712444, 157065267, 546145630, 68714550, 939433993, 742686779, 242561084, 283967550, 379052096, 367968322, 800456772, 861384775, 235114569, 121806922, 377901136, 119926869, 218439766, 800981689, 244600920, 314339417, 163958799, 859103325, 187936863, 492408930, 955502694, 342192231, 985120873, 829767786, 98857070, 895516783, 101400688, 159965297, 422637683, 105865333, 162017398, 167617087, 885989498, 935448699, 833429630, 181272713, 105140362, 298713229, 312324238, 104079504, 885133463, 165159065, 360181915, 923681478, 939057318, 929796263, 237928616, 223486124, 333914287, 239472818, 487846071, 162081482, 514220222, 116871367, 895381707, 952632696, 169021655, 335220952, 257503461, 221274342, 228970729, 194228463, 756224243, 755388661, 490250488, 531509972, 754963839, 267